# Moloco Data Scientist Interview 

Importing the libraries 

In [1]:
import pandas as pd

Load the data 

In [2]:
df=pd.read_csv('Analytics.csv')

In [3]:
df.columns

Index(['ts', 'user_id', 'country_id', 'site_id'], dtype='object')

### Question 1

1. Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

In [4]:
df1=df[df['country_id']=='BDV']

In [5]:
df1['site_id'].value_counts()

5NPAU    717
N0OTG    122
3POLC      5
Name: site_id, dtype: int64

In [6]:
df2=df1.drop_duplicates(subset='user_id')

In [7]:
df2['site_id'].value_counts()

5NPAU    538
N0OTG     83
3POLC      2
Name: site_id, dtype: int64

- created a filter to get only website with the country_id as 'BDV' 
- selected the site_id 
- dropped the dublicates user_id for each site_id. 
- value_count method was used to count the number of unique user in each site. as seen above, 5NPAU has the highest number of users with 538 users  

### Question 2

2. Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [8]:
df['ts1']=pd.to_datetime(df['ts'])

In [9]:
df3=df[(df['ts1']>'2019-02-03 00:00:00')&(df['ts1']<'2019-02-04 23:59:59')]

In [10]:
df3['number of visits']=1

/Users/whitehouse/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df4=df3.groupby(["user_id","site_id"],as_index=False)["number of visits"].sum()

In [12]:
df4[df4['number of visits']>10].reset_index()

,index,user_id,site_id,number of visits
0,3,LC06C3,N0OTG,25
1,417,LC3A59,N0OTG,26
2,485,LC3C7E,3POLC,15
3,493,LC3C9D,N0OTG,17


- changed the column "ts" to datetime. 
- created a filter for the date between ('2019-02-03 00:00:00')&('2019-02-04 23:59:59'). 
- created a column "number of visits" equal to 1.
- then grouped by "user_id" and "site_id" and summed up the number of visits with the sum method.
- created another filter to show only visits more than 10. 
Then, I got the result as shown above.

### Question 3

For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

In [13]:
user=list(df['user_id'].unique())

In [14]:
con=[]
for i in user:
    df8=df[df['user_id']==str(i)]
    a=max(df8['ts1'])
    i=df[(df['ts1']==a)&(df['user_id']==i)]
    con.append(i)

In [15]:
df_1=pd.concat(con)

In [16]:
df_1['number of users']=1

In [17]:
df_1=df_1.groupby(["site_id"],as_index=False)["number of users"].count().sort_values("number of users", ascending = False)

In [18]:
df_1.reset_index(inplace=True)

In [19]:
df_1.columns

Index(['index', 'site_id', 'number of users'], dtype='object')

In [20]:
del df_1['index']

In [21]:
df_1.head(3)

,site_id,number of users
0,5NPAU,992
1,N0OTG,561
2,QGO3G,289


- Created a string of unique user_id.
- Created a for-loop to find the last visited site by the unique user_id and appended that row to the data con. 
- Used the count function on df_1 to count the number of user_id by the site_id and got the result as shown above.

### Question 4

For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [22]:
last=[]
first=[]
uses=[]
for i in user:
    df8=df[df['user_id']==str(i)]
    a=max(df8['ts1'])
    b=min(df8['ts1'])
    iw=df[(df['ts1']==a)&(df['user_id']==i)]
    ir=df[(df['ts1']==b)&(df['user_id']==i)]
    f_site=list(ir['site_id'])
    l_site=list(iw['site_id'])
    for r,v in zip(l_site,f_site):
        last.append(r)
        first.append(v)
        uses.append(i)

In [23]:
compare=[]
for i,v in zip(last,first):
    if i==v:
        a='TRUE'
    else:
        a='FALSE'
    compare.append(a)

In [24]:
site=pd.DataFrame({
    'user_id':uses,
    'last_site_id':last,
    'first_site_id':first,
    'compare':compare
})

In [25]:
user_true=list(site[site['compare']=='TRUE']['user_id'])

In [26]:
len(user_true)

1670

In [27]:
site.head()

,user_id,last_site_id,first_site_id,compare
0,LC36FC,N0OTG,N0OTG,TRUE
1,LC39B6,N0OTG,N0OTG,TRUE
2,LC3500,N0OTG,N0OTG,TRUE
3,LC374F,N0OTG,N0OTG,TRUE
4,LCC1C3,QGO3G,QGO3G,TRUE


Used the string of the users that was created in the previous question 
created a loop to get the first and last site_id using datestamps per unique user 
Created another loop to compare the first site_id to the last site_id 
Further created an object of the true condition and a length of the object 